In [1]:
import requests
import time
import csv
import random

In [2]:
categories = {"categories":[{"cat_key":"OVERALL","name":"Overall"},{"cat_key":"APPLICATION","name":"All apps"},{"cat_key":"GAME","name":"All games"},{"cat_key":"ART_AND_DESIGN","name":"Art & Design"},{"cat_key":"AUTO_AND_VEHICLES","name":"Auto & Vehicles"},{"cat_key":"BEAUTY","name":"Beauty"},{"cat_key":"BOOKS_AND_REFERENCE","name":"Books & Reference"},{"cat_key":"BUSINESS","name":"Business"},{"cat_key":"COMICS","name":"Comics"},{"cat_key":"COMMUNICATION","name":"Communication"},{"cat_key":"DATING","name":"Dating"},{"cat_key":"EDUCATION","name":"Education"},{"cat_key":"ENTERTAINMENT","name":"Entertainment"},{"cat_key":"EVENTS","name":"Events"},{"cat_key":"FINANCE","name":"Finance"},{"cat_key":"FOOD_AND_DRINK","name":"Food & Drink"},{"cat_key":"HEALTH_AND_FITNESS","name":"Health & Fitness"},{"cat_key":"HOUSE_AND_HOME","name":"House & Home"},{"cat_key":"LIFESTYLE","name":"Lifestyle"},{"cat_key":"MAPS_AND_NAVIGATION","name":"Maps & Navigation"},{"cat_key":"MEDICAL","name":"Medical"},{"cat_key":"MUSIC_AND_AUDIO","name":"Music & Audio"},{"cat_key":"NEWS_AND_MAGAZINES","name":"News & Magazines"},{"cat_key":"PARENTING","name":"Parenting"},{"cat_key":"PERSONALIZATION","name":"Personalization"},{"cat_key":"PHOTOGRAPHY","name":"Photography"},{"cat_key":"PRODUCTIVITY","name":"Productivity"},{"cat_key":"SHOPPING","name":"Shopping"},{"cat_key":"SOCIAL","name":"Social"},{"cat_key":"SPORTS","name":"Sports"},{"cat_key":"TOOLS","name":"Tools"},{"cat_key":"TRAVEL_AND_LOCAL","name":"Travel & Local"},{"cat_key":"VIDEO_PLAYERS","name":"Video Players & Editors"},{"cat_key":"WEATHER","name":"Weather"},{"cat_key":"LIBRARIES_AND_DEMO","name":"Libraries & Demo"},{"cat_key":"GAME_ARCADE","name":"Arcade"},{"cat_key":"GAME_PUZZLE","name":"Puzzle"},{"cat_key":"GAME_CARD","name":"Cards"},{"cat_key":"GAME_CASUAL","name":"Casual"},{"cat_key":"GAME_RACING","name":"Racing"},{"cat_key":"GAME_SPORTS","name":"Sport Games"},{"cat_key":"GAME_ACTION","name":"Action"},{"cat_key":"GAME_ADVENTURE","name":"Adventure"},{"cat_key":"GAME_BOARD","name":"Board"},{"cat_key":"GAME_CASINO","name":"Casino"},{"cat_key":"GAME_EDUCATIONAL","name":"Educational"},{"cat_key":"GAME_MUSIC","name":"Music Games"},{"cat_key":"GAME_ROLE_PLAYING","name":"Role Playing"},{"cat_key":"GAME_SIMULATION","name":"Simulation"},{"cat_key":"GAME_STRATEGY","name":"Strategy"},{"cat_key":"GAME_TRIVIA","name":"Trivia"},{"cat_key":"GAME_WORD","name":"Word Games"},{"cat_key":"ANDROID_WEAR","name":"Android Wear"},{"cat_key":"FAMILY","name":"Family All Ages"},{"cat_key":"FAMILY_UNDER_5","name":"Family Ages 5 & Under"},{"cat_key":"FAMILY_6_TO_8","name":"Family Ages 6-8"},{"cat_key":"FAMILY_9_AND_UP","name":"Family Ages 9 & Up"},{"cat_key":"FAMILY_ACTION","name":"Family Action"},{"cat_key":"FAMILY_BRAINGAMES","name":"Family Brain Games"},{"cat_key":"FAMILY_CREATE","name":"Family Create"},{"cat_key":"FAMILY_EDUCATION","name":"Family Education"},{"cat_key":"FAMILY_MUSICVIDEO","name":"Family Music & Video"},{"cat_key":"FAMILY_PRETEND","name":"Family Pretend Play"}]}
category_keys = [cat["cat_key"] for cat in categories["categories"]]

In [3]:
def get_top_free_applications(categories):
    def per_category_top_free(category):
        waiting_urls = set()
        url = 'http://localhost:3000/api/apps/?collection=topselling_free&category={}&lang=en'.format(category)
        #top free applications
        response = requests.get(url)
        if response.status_code == 200:
            for app in response.json()["results"]:
                waiting_urls.add(app["url"])
        else:
            print("Request Error")
        return waiting_urls
    all_top_free_apps = set()
    for category in categories:
        all_top_free_apps.update(per_category_top_free(category))
    return all_top_free_apps

In [4]:
def get_app_category(app_id):
    base_url = 'http://localhost:3000/api/apps/'
    response = requests.get(base_url+app_id)
    if response.status_code == 200:
        gid = response.json()["genreId"].upper()
        return gid
    else:
        print(app_id, " category cannot be found")
        return "NOT_FOUND"

In [7]:
def get_app_ids(top_free_apps):
    waiting_urls = list(top_free_apps)
    random.shuffle(waiting_urls)
    waiting_urls_unique_check = set([url for url in waiting_urls])
    app_ids = set()

    
    counter = 0
    start_time = time.time()
    while (len(app_ids) + len(waiting_urls)) < 100000:
        if counter % 10 == 0:
            elapsed_time = time.time() - start_time
            print("Total number of apps", len(app_ids) + len(waiting_urls))
            print("Collected app ids", len(app_ids))
            print("Elapsed time up to now is {}".format(elapsed_time))
        try:
            url = waiting_urls.pop()
            waiting_urls_unique_check.remove(url)
        except Exception:
            print("All linked applications are traversed")
            break

        #add apk id if it is free, popular, and has longer description than 100 characters
        response = requests.get(url)
        if response.status_code == 200:
            json = response.json()
            if json["minInstalls"] > 10000 and json["priceText"] == "Free" and len(json["description"]) > 500:
                if url.split('/')[-1] not in app_ids:
                    response = requests.get(url+"/permissions")
                    if response.status_code == 200:
                        json = response.json()
                        exist = False
                        for line in json["results"]:
                            if "record audio" in line["permission"]:
                                app_ids.add(url.split('/')[-1] )
                                application_id = url.split('/')[-1]
                                cat = get_app_category(application_id)
                                with open("READ_CONTACTS/read_contacts_{}.txt".format(cat), "a") as target:
                                    target.write(application_id + "\n")
                                

        #add similar app urls
        response = requests.get(url + '/similar')
        if response.status_code == 200:
            for app in response.json()["results"]:
                if app["appId"] not in app_ids:
                    if app["url"] not in waiting_urls_unique_check:
                        waiting_urls_unique_check.add(app["url"])
                        waiting_urls.append(app["url"])
        random.shuffle(waiting_urls)
        counter += 1

    for url in waiting_urls:
        response = requests.get(url)
        if response.status_code == 200:
            json = response.json()
            if json["minInstalls"] > 10000 and json["priceText"] == "Free" and len(json["description"]) > 500:
                if url.split('/')[-1] not in app_ids:
                    app_ids.add(url.split('/')[-1] )
                    cat = get_app_category(application_id)
                    with open("READ_CONTACTS/read_contacts_{}.txt".format(cat), "a") as target:
                        target.write(application_id + "\n")

In [6]:
top_free_apps = get_top_free_applications(category_keys)
get_app_ids(top_free_apps)

Request Error
Request Error
Request Error
Request Error
Total number of apps 3173
Collected app ids 0
Elapsed time up to now is 1.9073486328125e-06
Total number of apps 3166
Collected app ids 3
Elapsed time up to now is 9.56020736694336
Total number of apps 3157
Collected app ids 4
Elapsed time up to now is 18.778782844543457
Total number of apps 3148
Collected app ids 5
Elapsed time up to now is 27.884185791015625
Total number of apps 3138
Collected app ids 5
Elapsed time up to now is 36.338220834732056
Total number of apps 3128
Collected app ids 5
Elapsed time up to now is 45.01338005065918
Total number of apps 3120
Collected app ids 7
Elapsed time up to now is 54.524980306625366
Total number of apps 3111
Collected app ids 8
Elapsed time up to now is 63.599225759506226
Total number of apps 3105
Collected app ids 12
Elapsed time up to now is 73.4266676902771
Total number of apps 3097
Collected app ids 14
Elapsed time up to now is 82.62183237075806
Total number of apps 3087
Collected a

Total number of apps 2419
Collected app ids 136
Elapsed time up to now is 818.8586416244507
Total number of apps 2413
Collected app ids 140
Elapsed time up to now is 828.4590177536011
Total number of apps 2404
Collected app ids 141
Elapsed time up to now is 837.6100332736969
Total number of apps 2395
Collected app ids 142
Elapsed time up to now is 846.8634440898895
Total number of apps 2386
Collected app ids 143
Elapsed time up to now is 855.4435744285583
Total number of apps 2376
Collected app ids 143
Elapsed time up to now is 863.818395614624
Total number of apps 2368
Collected app ids 145
Elapsed time up to now is 872.9686427116394
Total number of apps 2358
Collected app ids 145
Elapsed time up to now is 882.0295021533966
Total number of apps 2348
Collected app ids 145
Elapsed time up to now is 890.34668135643
Total number of apps 2341
Collected app ids 148
Elapsed time up to now is 899.775887966156
Total number of apps 2331
Collected app ids 148
Elapsed time up to now is 908.103900

Total number of apps 1667
Collected app ids 274
Elapsed time up to now is 1639.4809920787811
Total number of apps 1658
Collected app ids 275
Elapsed time up to now is 1648.7784979343414
Total number of apps 1648
Collected app ids 275
Elapsed time up to now is 1657.1465830802917
Total number of apps 1640
Collected app ids 277
Elapsed time up to now is 1666.5079448223114
Total number of apps 1630
Collected app ids 277
Elapsed time up to now is 1675.007597208023
Total number of apps 1621
Collected app ids 278
Elapsed time up to now is 1683.397588968277
Total number of apps 1613
Collected app ids 280
Elapsed time up to now is 1692.6749377250671
Total number of apps 1603
Collected app ids 280
Elapsed time up to now is 1701.9004757404327
Total number of apps 1594
Collected app ids 281
Elapsed time up to now is 1710.4160914421082
Total number of apps 1587
Collected app ids 284
Elapsed time up to now is 1720.2854158878326
Total number of apps 1577
Collected app ids 284
Elapsed time up to now i

Total number of apps 893
Collected app ids 390
Elapsed time up to now is 2444.5036573410034
Total number of apps 884
Collected app ids 391
Elapsed time up to now is 2452.921647787094
Total number of apps 877
Collected app ids 394
Elapsed time up to now is 2462.870019197464
Total number of apps 869
Collected app ids 396
Elapsed time up to now is 2472.5761506557465
Total number of apps 859
Collected app ids 396
Elapsed time up to now is 2481.266999721527
Total number of apps 849
Collected app ids 396
Elapsed time up to now is 2489.28724861145
Total number of apps 842
Collected app ids 399
Elapsed time up to now is 2499.0823998451233
Total number of apps 834
Collected app ids 401
Elapsed time up to now is 2508.9392771720886
Total number of apps 829
Collected app ids 406
Elapsed time up to now is 2519.2511870861053
Total number of apps 820
Collected app ids 407
Elapsed time up to now is 2528.0464651584625
Total number of apps 812
Collected app ids 409
Elapsed time up to now is 2537.2467010

IndexError: pop from empty list

In [ ]:
"""
from threading import Thread, Condition
import time
import random
from collections import defaultdict


waiting_urls = list(top_free_apps)
random.shuffle(waiting_urls)
waiting_urls_unique_check = set([url for url in waiting_urls])
app_ids = set()
start_time = time.time()
similarity_candidates = []
condition = Condition()
categories = {}

class SimilarityThread(Thread):
    def run(self):
        nums = range(5)
        global waiting_urls, waiting_urls_unique_check, similarity_candidates
        while True:
            condition.acquire()
            random.shuffle(similarity_candidates)
            for i in range(3):
                url = similarity_candidates.pop()
                response = requests.get(url + '/similar')
                if response.status_code == 200:
                    for app in response.json()["results"]:
                        if app["appId"] not in app_ids:
                            if app["url"] not in waiting_urls_unique_check:
                                waiting_urls_unique_check.add(app["url"])
                                waiting_urls.append(app["url"])
                else:
                    print("SimilarityThread ", url)
            condition.notify()
            condition.release()
            time.sleep(random.random())


class AppIdThread(Thread):
    def run(self):
        global waiting_urls, similarity_candidates, app_ids, start_time, categories
        while True:
            condition.acquire()
            if not waiting_urls:
                print ("Nothing in queue, consumer is waiting")
                condition.wait()
                print ("Producer added something to queue and notified the consumer")
            
            elapsed_time = time.time() - start_time
            print("Total number of apps", len(app_ids) + len(waiting_urls))
            print("Collected app ids", len(app_ids))
            print("Elapsed time up to now is {}".format(elapsed_time))
            
           
            while waiting_urls:
                url = waiting_urls.pop()
                waiting_urls_unique_check.remove(url)
                #add apk id if it is free, popular, and has longer description than 100 characters
                response = requests.get(url)
                if response.status_code == 200:
                    json = response.json()
                    if json["minInstalls"] > 10000 and json["priceText"] == "Free" and len(json["description"]) > 500:
                        if url.split('/')[-1] not in app_ids:
                            response = requests.get(url+"/permissions")
                            if response.status_code == 200:
                                json = response.json()
                                exist = False
                                for line in json["results"]:
                                    if "read your contacts" in line["permission"]:
                                        app_ids.add(url.split('/')[-1] ) 
                                        similarity_candidates.append(url)
                                        application_id = url.split('/')[-1]
                                        cat = get_app_category(application_id)
                                        with open("read_contacts_{}.txt".format(cat), "a") as target:
                                            target.write(application_id + "\n")
                else:
                    print("AppIdThread ", url)
                                    
            condition.release()
            time.sleep(random.random())
    
    
AppIdThread().start()
SimilarityThread().start()
"""

In [11]:
import play_scraper
print(play_scraper.similar('com.supercell.clashofclans', results=5))

TypeError: similar() got an unexpected keyword argument 'results'